In [4]:
import numpy as np
import pandas as pd
import os
import re
# import emoji

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



In [5]:

df=pd.read_csv('nlptrain.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


# Missing Values

In [12]:
def missingValues(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percentage=(total/len(data))
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

missingValues(df)

,Total,Percentage
selected_text,1,0.000036
text,1,0.000036
sentiment,0,0.000000
textID,0,0.000000


In [13]:
df=df.dropna()

In [26]:
def countValues(df,col):
    counts=df[col].value_counts().dropna()
    percentage=round(counts*100/np.sum(counts.values),2)
    return pd.concat([counts,percentage],axis=1,keys=['Total','Percentage'])
    
countValues(df,'sentiment')

,Total,Percentage
neutral,11117,40.45
positive,8582,31.23
negative,7781,28.32


In [33]:
def uniqueValues(df,col):
    uniques=np.unique(df[col],return_counts=True)
    return pd.DataFrame({'Title':uniques[0],
                         'Counts':uniques[1]})


uniqueValues(df,'sentiment')

,Title,Counts
0,negative,7781
1,neutral,11117
2,positive,8582


In [42]:
def duplicateValues(df):
    dups=[]
    columns=df.columns
    for i in df.columns:
        dups.append(sum(df[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dups)],axis=1,
                    keys=['Columns','Counts'])
duplicateValues(df)

,Columns,Counts
0,textID,0
1,text,0
2,selected_text,5017
3,sentiment,27477


In [43]:
df.describe()

,textID,text,selected_text,sentiment
count,27480,27480,27480,27480
unique,27480,27480,22463,3
top,81094a0fb7,yep,good,neutral
freq,1,1,199,11117


In [73]:
def find_url(string):
    text=re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return ''.join(text)
    
    
find_url('I love spending time at https://www.kaggle.com/')

'https://www.kaggle.com/'

In [74]:
df['url']=df['text'].apply(lambda x: find_url(x))

In [79]:
df.head(10)['url']

0                                  
1                                  
2                                  
3                                  
4                                  
5    http://www.dothebouncy.com/smf
6                                  
7                                  
8                                  
9                                  
Name: url, dtype: object